Spent a bit of time testing out a GRU model, but ran out of time... just posting what I have so far, will run, but not on Kaggle.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader #doesn't work on kaggle
import torch
import torch.nn as nn
from torch.autograd import Variable
import time, math
import random

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

# DataLoader

In [ ]:
one_year = pd.date_range(pd.to_datetime('2016-08-15'),pd.to_datetime('2017-08-15'))
cols = ['date','unit_sales','onpromotion']
df_to_copy = pd.DataFrame(index=one_year)
df_to_copy['day'] = df_to_copy.index.day
df_to_copy['month'] = df_to_copy.index.month

class GroceryDataset(Dataset):
    
    #Create a fixed one-year length time series of sales data
    def groups_to_series(self, x):
        global df_to_copy
        df = df_to_copy.copy()
        df = df.merge(x[cols], right_on='date', left_index=True, how='left')
        df = df.fillna(0)
        df = df.as_matrix()
        return df
    def __init__(self):
        
        
        #load different data
        self.train = pd.read_csv('data/grocery kaggle comp/train 2.csv')
        self.train.date = pd.to_datetime(self.train.date)
        self.train = self.train[self.train.date>pd.to_datetime('2016-08-14')] #I used one year of data for training
        self.train = self.train.drop('id',axis=1)
        self.train.onpromotion = self.train.onpromotion.astype(int)
        self.train.unit_sales = self.train.unit_sales.apply(abs)
        
        #item, store combinations
        self.group = list(self.train.groupby(['item_nbr','store_nbr']).groups.keys())
        

        
        self.item_dataset = pd.read_csv('data/grocery kaggle comp/items.csv').set_index('item_nbr')
        self.store_dataset = pd.read_csv('data/grocery kaggle comp/stores.csv').set_index('store_nbr')
        self.len = len(self.group)
        
        self.item_set = list(set(self.train['item_nbr']))
        self.store_set = list(set(self.train['store_nbr']))
        self.fam_set = list(set(self.item_dataset['family']))
        self.cluster_set = list(set(self.store_dataset['cluster']))
        
        self.item_set_len = len(self.item_set)
        self.store_set_len = len(self.store_set)
        self.fam_set_len = len(self.fam_set)
        self.cluster_set_len = len(self.cluster_set)

        #onvert values to categories
        self.item_to_ix = {var: i for i, var in enumerate(self.item_set)}
        self.store_to_ix = {var: i for i, var in enumerate(self.store_set)}
        self.fam_to_ix = {var: i for i, var in enumerate(self.fam_set)}
        self.cluster_to_ix = {var: i for i, var in enumerate(self.cluster_set)}
    
        
    def __getitem__(self, ind):
                
        #for a particular item and store, create the one years data series
        item, store = self.group[ind]
        
        data = self.train[(self.train.store_nbr==store)&(self.train.item_nbr==item)].groupby(['store_nbr','item_nbr']).apply(self.groups_to_series)

        
        fam = self.item_dataset.loc[item].family
        cluster = self.store_dataset.loc[store].cluster
        
        #categorize other data
        item_tensor = torch.LongTensor([[self.item_to_ix[item]]])
        
        store_tensor = torch.LongTensor([[self.store_to_ix[store]]])
        
        fam_tensor = torch.LongTensor([[self.fam_to_ix[fam]]])
        
        cluster_tensor = torch.LongTensor([[self.cluster_to_ix[cluster]]])
                                
        unit_sales = torch.from_numpy(np.array([x[3:] for x in data.iloc[0][:-1]]).astype(float)).type(torch.FloatTensor)
        
        day = torch.from_numpy(np.array([x[0] for x in data.iloc[0][:-1]])).type(torch.LongTensor)
        
        month = torch.from_numpy(np.array([x[1] for x in data.iloc[0][:-1]])).type(torch.LongTensor)
        
        y = torch.from_numpy(np.array([x[3] for x in data.iloc[0][1:]])).type(torch.FloatTensor)
        
        return item_tensor, store_tensor,fam_tensor, cluster_tensor, unit_sales, day, month, y
    def __len__(self):
        return self.len

In [ ]:
dataset = GroceryDataset()
train_loader = DataLoader(dataset= dataset, batch_size=32, shuffle=False, num_workers=0)

# GRU Model

In [ ]:
class RNN(nn.Module):
    def __init__(self, bs,input_size, item_set_size ,store_set_size,fam_set_size,
                 cluster_set_size, hidden_size, output_size, n_layers=1):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.batch_size = bs
        
        self.item_encoder = nn.Embedding(item_set_size, 40)
        self.store_encoder = nn.Embedding(store_set_size, 5)
        self.fam_encoder = nn.Embedding(fam_set_size, 5)
        self.cluster_encoder = nn.Embedding(cluster_set_size, 5)  
        self.day_encoder = nn.Embedding(32, 10)
        self.month_encoder = nn.Embedding(13, 3)
        
        self.gru = nn.GRU(input_size+40+5+5+5+10+3, hidden_size, n_layers, batch_first=True, dropout=0.2)
        self.regressor = nn.Linear(hidden_size, output_size)
    
    def forward(self, input, item_tensor, store_tensor,fam_tensor, cluster_tensor, day, month, hidden):

        embedding = torch.cat((self.item_encoder(item_tensor.squeeze()), 
                                self.day_encoder(day),
                                self.month_encoder(month),
                                self.store_encoder(store_tensor.squeeze()), self.fam_encoder(fam_tensor.squeeze()),
                                self.cluster_encoder(cluster_tensor.squeeze())),1)
        
        
        input = torch.cat((input, embedding),1).unsqueeze(1)

        output, hidden = self.gru(input,hidden)
        output = self.regressor(output)
        return output, hidden

    def init_hidden(self):
        return Variable(torch.zeros(self.n_layers,self.batch_size,self.hidden_size))

In [ ]:
n_epochs = 100
hidden_size = 64
output_size = 1
n_layers = 2
lr = 0.005
batch_size=32

decoder = RNN(batch_size,2, dataset.item_set_len, dataset.store_set_len, dataset.fam_set_len,dataset.cluster_set_len, hidden_size , output_size, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.MSELoss() #Not using proper loss function

# Train

In [ ]:
loss_avg = 0


def time_since(since):
    s = time.time() - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

start = time.time()
for i, data in enumerate(train_loader, 0):
    
    item_tensor, store_tensor,fam_tensor, cluster_tensor, unit_sales, day, month, y = data
    item = Variable(item_tensor, requires_grad=False)
    store = Variable(store_tensor,requires_grad=False)
    fam = Variable(fam_tensor,requires_grad=False)
    cluster = Variable(cluster_tensor,requires_grad=False) 
    d = Variable(day,requires_grad=False) #day of week
    m = Variable(month,requires_grad=False) # month of year
    row = Variable(unit_sales,requires_grad=False) #sales for that day, and if on promo
    
    y = Variable(y,requires_grad=False)
    
    hidden = decoder.init_hidden()
    decoder.zero_grad()
    
    outputs = Variable(torch.zeros((len(row[0]), row.size()[0]))) #series to collect ouputs
    loss = 0
    force = random.random() < 0.5
    
    #teacher-forcing or not
    if force:
        for c in range(len(row[0])):
            outputs[c], hidden = decoder(row[:,c],item, store,fam , cluster, d[:,c],m[:,c] , hidden)
    else:
         for c in range(len(row[0])):
            if c>0:
                outputs[c], hidden = decoder(torch.cat((outputs[c-1].unsqueeze(1),row[:,c,1].unsqueeze(1)),1),item, store,fam , cluster, d[:,c],m[:,c] , hidden)   
            else:
                outputs[c], hidden = decoder(row[:,c],item, store,fam , cluster, d[:,c],m[:,c] , hidden)

    loss += criterion(outputs[15:].view(32,-1),y[:,15:]) #hard coded batch-size... only eval after 15 time steps
    loss.backward()
    decoder_optimizer.step()
    

    if i % 2 == 0:
        print('[%s (%d %d%%) %.4f]' % (time_since(start), i, i / 10, loss.data[0]/y.size()[1]))
        #print(evaluate('Wh', 100), '\n')
